# 属性描述符(精通Python的关键)

In [1]:
class Quantity:
    
    def __init__(self, storage_name):
        self.storage_name = storage_name    # storage_name为描述符实例的实例属性
        
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
            # 这里的instance为托管实例，self为描述符实例，self.storage_name为托管实例的储存属性，存储自身托管属性
        else:
            raise ValueError('value must be >0')
            

class LineItem:
    
    weight = Quantity('weight')
    price = Quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [2]:
truffle = LineItem('White truffle', 100, 0)

ValueError: value must be >0

In [10]:
truffle = LineItem('White truffle', 100, 15)

In [11]:
truffle.__dict__

{'_NonBlank#0': 'White truffle', '_Quantity#0': 100, '_Quantity#1': 15}

In [12]:
truffle.weight

100

In [13]:
truffle.price

15

In [14]:
class Quantity:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
        
    def __get__(self, instance, owner):    # owner参数是托管类的引用，通过描述符从托管类中获取托管属性时用得到
        return getattr(instance, self.storage_name)

    
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('must be > 0')
            
            
class LineItem:
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [15]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)

In [16]:
coconuts.weight, coconuts.price

(20, 17.95)

In [17]:
coconuts.__dict__

{'description': 'Brazilian coconut', '_Quantity#0': 20, '_Quantity#1': 17.95}

In [18]:
getattr(coconuts, '_Quantity#0'), getattr(coconuts, '_Quantity#1')

(20, 17.95)

In [19]:
LineItem.weight    # 从类中获取托管属性，而不是从实例中获取托管属性时，描述符的__get__方法接收到的托管实例参数instance为None

AttributeError: 'NoneType' object has no attribute '_Quantity#0'

In [22]:
class Quantity:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('must be > 0')    # 验证属性，当传入的price参数小于等于0时，抛出异常
            
            
class LineItem:
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight    # 调用描述符实例的__set__方法
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [23]:
LineItem.price

In [24]:
br_nuts = LineItem('Brazil nuts', 10, 34.95)

In [25]:
br_nuts.price

34.95

In [26]:
br_nuts.__dict__

{'description': 'Brazil nuts', '_Quantity#0': 10, '_Quantity#1': 34.95}

In [27]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)

In [28]:
coconuts.__dict__

{'description': 'Brazilian coconut', '_Quantity#0': 20, '_Quantity#1': 17.95}

In [29]:
def quantity():
    try:
        quantity.counter += 1
    except AttributeError:
        quantity.counter = 0
    
    storage_name = '_{}:{}'.format('quantity', quantity.counter)
    
    def qty_getter(instance):
        return getattr(instance, storage_name)
    
    def qty_setter(instance, value):
        if value > 0:
            setattr(instance, storage_name, value)
        else:
            raise ValueError('value must be > 0')
            
    return property(qty_getter, qty_setter)


class LineItem:
    weight = quantity()
    price = quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [30]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)

In [31]:
coconuts.__dict__

{'description': 'Brazilian coconut', '_quantity:0': 20, '_quantity:1': 17.95}

## 一种新型描述符

In [119]:
import abc


class AutoStorage:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
        
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)
        
    def __delete__(self, instance):
        delattr(instance, self.storage_name)
            
            
class Validated(abc.ABC, AutoStorage):
    
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
        
    @abc.abstractmethod
    def validate(self, instance, value):
        """return validated value or raise ValueError"""
        
        
class Quantity(Validated):
    """a number greater than zero"""
    
    def validate(self, instance, value):
        if value <= 0 :
            raise ValueError('must be > 0')
        else:
            return value
        

class NonBlank(Validated):
    """a string with at least one non-space character"""
    
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value
    
    
class LineItem:
    description = NonBlank()    # NonBlank的_AutoStorage__counter属性会变为1
    weight = Quantity()
    price = Quantity()          # Quantity的_AutoStorage__counter属性会变为2
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [125]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)

In [126]:
vars(coconuts)

{'_NonBlank#0': 'Brazilian coconut', '_Quantity#0': 20, '_Quantity#1': 17.95}

In [127]:
coconuts = LineItem(' ', 20, 17.95)

ValueError: value cannot be empty or blank

In [128]:
del coconuts.weight

In [129]:
coconuts.weight

AttributeError: 'LineItem' object has no attribute '_Quantity#0'

In [130]:
vars(coconuts)

{'_NonBlank#0': 'Brazilian coconut', '_Quantity#1': 17.95}

In [36]:
NonBlank._AutoStorage__counter

1

In [37]:
Quantity._AutoStorage__counter

2

In [38]:
coconuts.__dict__['weight'] = 27

In [39]:
coconuts.__dict__

{'_NonBlank#0': 'Brazilian coconut',
 '_Quantity#0': 20,
 '_Quantity#1': 17.95,
 'weight': 27}

In [40]:
coconuts.weight

20

In [41]:
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]


def display(obj):
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return '<{} object>'.format(cls_name(obj))


def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))

In [42]:
class Overriding:
    """也成数据描述符或强制描述符"""
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)
        
    def __set__(self, instance, value):
        print_args('set', self, instance, value)
        

class OverridingNoGet:
    """没有``__get__``方法的覆盖型描述符"""
    def __set__(self, instance, value):
        print_args('set', self, instance, value)
        
        
class NonOverridng:
    """也称非数据描述符或遮盖型描述符"""
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)   
        
        
class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverridng()
    
    def spam(self):
        print('-> Managed.spam({})'.format(display(self)))

### 覆盖型描述符

In [43]:
obj = Managed()

In [44]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [45]:
Managed.over

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [46]:
obj.over = 7

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [47]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [48]:
obj.__dict__['over'] = 8

In [49]:
vars(obj)

{'over': 8}

In [50]:
obj.over    # 简单来说就是从托管实例中获取托管属性会覆盖对实例属性的读取操作，因为这个操作会调用描述符的__get__方法

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


### 没有\__get__方法的覆盖型描述符

In [51]:
obj.over_no_get

In [52]:
Managed.over_no_get

In [53]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [54]:
obj.over_no_get    # 上一步的赋值操作并没有在同名的储存属性中存储值，读取托管属性时也没有__get__方法，因此返回描述符实例本身

In [55]:
obj.__dict__['over_no_get'] = 9

In [56]:
obj.over_no_get

9

In [57]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [58]:
obj.over_no_get    
# 因为从托管实例读取托管属性时，描述符没有__get__方法，而托管实例具有与托管属性同名的实例属性，所以就会遮盖托管属性的读取操作，
# 返回实例属性而不是描述符实例本身

9

### 非覆盖型描述符

In [59]:
obj.non_over

-> NonOverridng.__get__(<NonOverridng object>, <Managed object>, <class Managed>)


In [60]:
obj.non_over = 7

In [61]:
obj.non_over    # 现在obj有个名为non_over的实例属性，把Managed类的同名托管属性non_over覆盖掉，因为这个描述符没有实现__set__方法

7

In [62]:
Managed.non_over

-> NonOverridng.__get__(<NonOverridng object>, None, <class Managed>)


In [63]:
del obj.non_over

In [64]:
obj.non_over

-> NonOverridng.__get__(<NonOverridng object>, <Managed object>, <class Managed>)


## 在类中覆盖描述符

In [65]:
obj = Managed()

In [66]:
Managed.over = 1

In [67]:
Managed.over_no_get = 2

In [68]:
Managed.non_over = 3

In [69]:
obj.over, obj.over_no_get, obj.non_over

(1, 2, 3)

### 绑定方法属于非覆盖型描述符

In [70]:
class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverridng()
    
    def spam(self):
        print('-> Managed.spam({})'.format(display(self)))

In [71]:
obj = Managed()

In [72]:
obj.spam    # 从托管实例中获取托管属性时，会触发描述符的__get__方法，获取的是绑定方法对象

<bound method Managed.spam of <__main__.Managed object at 0x7f90302aa280>>

In [73]:
Managed.spam    # 从托管类获取托管属性时，会触发描述符的__get__方法，该方法收到的托管实例参数instance为None。
# 所以从托管类中获取托管属性时，返回的是描述符实例本身，也就是返回函数自身的引用。

<function __main__.Managed.spam(self)>

In [74]:
obj.spam = 7    # 由于绑定方法是非覆盖型描述符，所以创建一个同名的实例属性会覆盖托管属性（描述符属性）

In [75]:
obj.spam

7

In [76]:
del obj.spam

In [77]:
obj.spam

<bound method Managed.spam of <__main__.Managed object at 0x7f90302aa280>>

In [78]:
import collections


class Text(collections.UserString):
    
    def __repr__(self):
        return 'Text({!r})'.format(self.data)
    
    def reverse(self):
        return self[::-1]

In [79]:
word = Text('forward')

In [80]:
word

Text('forward')

In [81]:
word.reverse()

Text('drawrof')

In [90]:
Text.reverse(Text('backward'))

Text('drawkcab')

In [91]:
type(Text.reverse), type(word.reverse)

(function, method)

In [92]:
list(map(Text.reverse, ['repaid', (10, 20, 30), Text('stressed')]))

['diaper', (30, 20, 10), Text('desserts')]

In [95]:
Text.reverse.__get__(word)

<bound method Text.reverse of Text('forward')>

In [96]:
Text.reverse.__get__(word, Text)

<bound method Text.reverse of Text('forward')>

In [97]:
Text.reverse.__get__(None, Text)

<function __main__.Text.reverse(self)>

In [98]:
word.reverse

<bound method Text.reverse of Text('forward')>

In [99]:
word.reverse.__self__

Text('forward')

In [100]:
word.reverse.__func__

<function __main__.Text.reverse(self)>

In [101]:
help(LineItem)

Help on class LineItem in module __main__:

class LineItem(builtins.object)
 |  LineItem(description, weight, price)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, description, weight, price)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  subtotal(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  description
 |      a string with at least one non-space character
 |  
 |  price
 |      a number greater than zero
 |  
 |  weight
 |      a number greater than zero



In [102]:
help(LineItem.weight)

Help on Quantity in module __main__:

    a number greater than zero



In [103]:
help(Quantity)

Help on class Quantity in module __main__:

class Quantity(Validated)
 |  a number greater than zero
 |  
 |  Method resolution order:
 |      Quantity
 |      Validated
 |      abc.ABC
 |      AutoStorage
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  validate(self, instance, value)
 |      return validated value or raise ValueError
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Validated:
 |  
 |  __set__(self, instance, value)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from Validated:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ---------------------------------

In [104]:
help(NonBlank)

Help on class NonBlank in module __main__:

class NonBlank(Validated)
 |  a string with at least one non-space character
 |  
 |  Method resolution order:
 |      NonBlank
 |      Validated
 |      abc.ABC
 |      AutoStorage
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  validate(self, instance, value)
 |      return validated value or raise ValueError
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Validated:
 |  
 |  __set__(self, instance, value)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from Validated:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  -------------